In [1]:
import semantic_kernel as sk

kernel = sk.Kernel()

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

api_key, org_id = sk.openai_settings_from_dot_env()
service_id = "default"
kernel.add_service(
    OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-0125", api_key=api_key, org_id=org_id),
)

plugin = kernel.import_plugin_from_prompt_directory("./Plugins/", "ReportPlugin")
OutlineJSON_function = plugin["OutlineJSON"]
OutlineJSON = await kernel.invoke(OutlineJSON_function, sk.KernelArguments(input="Báo cáo thị trường Tiêu dùng năm 2023"))
print(OutlineJSON)



{'report': {'title': 'Báo cáo thị trường Tiêu dùng năm 2023','introduction': 'Trong phần giới thiệu, báo cáo sẽ trình bày tình hình thị trường tiêu dùng hiện tại và những xu hướng dự đoán cho năm 2023.','body': {'headings': ['Tình hình thị trường tiêu dùng hiện tại','Xu hướng tiêu dùng trong năm 2023','Ảnh hưởng của yếu tố ngoại cảnh đến thị trường tiêu dùng','Phân tích sự cạnh tranh trong ngành tiêu dùng']}, 'conclusion': 'Tóm lại, báo cáo đã trình bày chi tiết về thị trường tiêu dùng năm 2023 và những yếu tố quan trọng liên quan.','recommendations': 'Dựa trên những phân tích trong báo cáo, chúng tôi đưa ra các khuyến nghị cụ thể cho các doanh nghiệp hoạt động trong lĩnh vực tiêu dùng để áp dụng trong năm 2023.'}}


In [2]:
from Plugins.utils import json_split
outline_dict = json_split(str(OutlineJSON))
outline_dict

{'title': 'Báo cáo thị trường Tiêu dùng năm 2023',
 'introduction': 'Trong phần giới thiệu, báo cáo sẽ trình bày tình hình thị trường tiêu dùng hiện tại và những xu hướng dự đoán cho năm 2023.',
 'body': {'headings': ['Tình hình thị trường tiêu dùng hiện tại',
   'Xu hướng tiêu dùng trong năm 2023',
   'Ảnh hưởng của yếu tố ngoại cảnh đến thị trường tiêu dùng',
   'Phân tích sự cạnh tranh trong ngành tiêu dùng']},
 'conclusion': 'Tóm lại, báo cáo đã trình bày chi tiết về thị trường tiêu dùng năm 2023 và những yếu tố quan trọng liên quan.',
 'recommendations': 'Dựa trên những phân tích trong báo cáo, chúng tôi đưa ra các khuyến nghị cụ thể cho các doanh nghiệp hoạt động trong lĩnh vực tiêu dùng để áp dụng trong năm 2023.'}

In [3]:
Introduction_function = plugin["Introduction"]
Introduction = await kernel.invoke(Introduction_function, sk.KernelArguments(introduction=outline_dict["introduction"]
                                                                            , reportLayout = str(OutlineJSON)))
print(Introduction)

Báo cáo thị trường Tiêu dùng năm 2023 sẽ cung cấp cái nhìn tổng quan về tình hình thị trường tiêu dùng hiện tại và những xu hướng được dự đoán cho năm 2023. Trước khi đi vào chi tiết về các phần còn lại của báo cáo, phần giới thiệu sẽ phân tích sự phát triển của thị trường tiêu dùng trong thời gian gần đây và những thay đổi dự kiến sẽ diễn ra trong năm tới. Điều này sẽ giúp độc giả hiểu rõ hơn về bối cảnh tổng thể của báo cáo và tạo nền tảng cho việc đánh giá chi tiết về các khía cạnh khác nhau của thị trường tiêu dùng.


In [4]:
from Plugins.google_search import GoogleSearchPlugin
from Plugins.utils import token_count
import asyncio
TOKEN_LIMIT = 8000

google_plugin = kernel.import_plugin_from_object(GoogleSearchPlugin(), plugin_name = "GoogleSearchPlugin")
google_search_function = google_plugin["GoogleSearch"]
get_content_function = google_plugin["GetContentFromURL"]

column_function = plugin["Column"]
column_summ_function = plugin["ColumnSumm"]
headings = outline_dict["body"]["headings"]

async def columnist(heading, title):
    search_result = []
    tokens = 0
    search_result_urls = await kernel.invoke(google_search_function, sk.KernelArguments(keyword = heading + f"{title}", num_results = 15))
    search_result_urls = search_result_urls.value
    print(search_result_urls)
    while True:
        if not search_result_urls:
            break
        url = search_result_urls.pop(0)
        content = await kernel.invoke(get_content_function, sk.KernelArguments(url = url))
        content = content.value
        new_tokens = token_count(content)
        if tokens + new_tokens > TOKEN_LIMIT:
            print(tokens)
            if tokens >= TOKEN_LIMIT//2:
                break
            else:
                continue
        tokens += new_tokens
        search_result.append(content)
    search_result = " ".join(search_result)
    column = await kernel.invoke(column_function, sk.KernelArguments(input = search_result, body_content = heading))
    column_summ = await kernel.invoke(column_summ_function, sk.KernelArguments(input = search_result, body_content = heading))
    return str(column), str(column_summ)

coros = []
for heading in headings:
    coros.append(columnist(heading, outline_dict["title"]))

values = await asyncio.gather(*coros)
print(values)




['https://innovativehub.com.vn/tong-quan-va-xu-huong-nganh-hang-tieu-dung-viet-nam-2023/', 'https://www.cimigo.com/vi/trends/xu-huong-tieu-dung-viet-nam-2023/', 'https://pmax.com.vn/blog/xu-huong-thi-truong-va-nguoi-tieu-dung-nua-cuoi-2023-dau-2024/#Du_doan_tinh_hinh_kinh_te_trong_nua_sau_nam_2023', 'https://aglobal.vn/blog/xu-huong-tieu-dung-quy-4-nam-2023-nhung-thay-doi-can-luu-y-1564161623', 'https://vneconomy.vn/xu-huong-tieu-dung-hien-dai-thau-hieu-de-chinh-phuc-thi-truong.htm', 'https://pmax.com.vn/blog/xu-huong-thi-truong-va-nguoi-tieu-dung-nua-cuoi-2023-dau-2024/#Xu_huong_thi_truong', 'https://mof.gov.vn/webcenter/portal/vclvcstc/pages_r/l/chi-tiet-tin?dDocName=MOFUCM262964', 'https://www.cimigo.com/vi/research-reports/bao-cao-xu-huong-tieu-dung-viet-nam-2023/', 'http://moit.gov.vn/tin-tuc/thi-truong-trong-nuoc/to-ng-mu-c-ba-n-le-ha-ng-ho-a-va-doanh-thu-di-ch-vu-tieu-du-ng-tang-9-6-.html', 'https://pmax.com.vn/blog/xu-huong-thi-truong-va-nguoi-tieu-dung-nua-cuoi-2023-dau-2024/#

In [5]:
for value in values:
    print(token_count(value[0]), "tokens")
    print(value[0])
    print(token_count(value[1]), "tokens")
    print(value[1])

848 tokens
Trong bối cảnh thị trường tiêu dùng hiện nay, việc nắm bắt và hiểu rõ tâm lý, hành vi của người tiêu dùng là vô cùng quan trọng để các doanh nghiệp có thể áp dụng các chiến lược tiếp thị hiệu quả. Việc đánh giá và dự đoán xu hướng thị trường cũng như tâm lý người tiêu dùng sẽ giúp các doanh nghiệp tạo ra các chiến dịch tiếp thị phù hợp và hiệu quả.

Trong thời kỳ khó khăn của nền kinh tế, người tiêu dùng thường có xu hướng cẩn trọng hơn trong việc chi tiêu. Họ có thể cắt giảm những khoản chi không cần thiết và tìm kiếm các sản phẩm, dịch vụ có giá trị thực sự. Do đó, các doanh nghiệp cần tập trung vào việc cải thiện chất lượng sản phẩm, tối ưu chi phí để đáp ứng nhu cầu của người tiêu dùng.

Xu hướng mua sắm thông minh và trải nghiệm mua hàng ngày càng trở nên phổ biến. Người tiêu dùng ngày càng thông minh hơn trong việc so sánh giá cả, tìm hiểu thông tin sản phẩm và đánh giá từ người tiêu dùng khác. Điều này đặt ra thách thức cho các doanh nghiệp trong việc cung cấp thông t

In [6]:
conclusion_function = plugin["Conclusion"]
summContent = " ".join([value[1] for value in values])
conclusion = await kernel.invoke(conclusion_function, sk.KernelArguments(conclusion = outline_dict["conclusion"], summContent = summContent))
print(str(conclusion))


Tóm lại, báo cáo đã trình bày chi tiết về thị trường tiêu dùng năm 2023 và những yếu tố quan trọng liên quan. Việc hiểu rõ xu hướng tiêu dùng và thích nghi với những biến động trong thị trường là vô cùng quan trọng để các doanh nghiệp có thể tồn tại và phát triển trong môi trường kinh doanh ngày càng cạnh tranh. Để cạnh tranh hiệu quả, các doanh nghiệp cần tập trung vào sáng tạo, nâng cao chất lượng sản phẩm và dịch vụ, cải thiện trải nghiệm khách hàng, áp dụng công nghệ và tự động hóa quy trình, cũng như quảng cáo và tiếp cận thông minh. Chỉ thông qua việc thực hiện những chiến lược này một cách hiệu quả, doanh nghiệp mới có thể đạt được sự thành công và bứt phá trong môi trường cạnh tranh ngày càng khốc liệt của ngành tiêu dùng.


In [7]:
print(token_count(str(conclusion)))

328


In [8]:
recommendation_function = plugin["Recommendation"]
summContent = " ".join([value[1] for value in values])
recommendation = await kernel.invoke(recommendation_function, sk.KernelArguments(recommendation = outline_dict["recommendations"], summContent = summContent))
print(str(recommendation))
print(token_count(str(recommendation)))

Dựa trên những phân tích trong báo cáo, chúng tôi đưa ra các khuyến nghị cụ thể cho các doanh nghiệp hoạt động trong lĩnh vực tiêu dùng để áp dụng trong năm 2023.

Đầu tiên, chúng tôi khuyến nghị các doanh nghiệp tập trung vào việc nâng cao chất lượng sản phẩm và dịch vụ. Việc đầu tư vào nghiên cứu và phát triển để cung cấp những sản phẩm mới, sáng tạo và hiện đại sẽ giúp doanh nghiệp thu hút khách hàng và tạo ra sự khác biệt trong thị trường. Đồng thời, việc đảm bảo sản phẩm đáp ứng các tiêu chuẩn về sức khỏe, an toàn thực phẩm và bền vững cũng là yếu tố quan trọng để thu hút người tiêu dùng hiện nay.

Thứ hai, chúng tôi khuyến nghị các doanh nghiệp tập trung vào trải nghiệm khách hàng. Đáp ứng nhanh chóng và hiệu quả nhu cầu của khách hàng, xây dựng mối quan hệ lâu dài và tạo ra sự hài lòng cho khách hàng sẽ giúp doanh nghiệp tạo ra sự tin tưởng và trung thành từ phía khách hàng. Việc cải thiện dịch vụ sau bán hàng và tạo ra các chương trình khuyến mãi hấp dẫn cũng là cách hiệu quả đ

In [9]:
for value in values:
    print(token_count(value[0]), "tokens")
    print(value[0])

848 tokens
Trong bối cảnh thị trường tiêu dùng hiện nay, việc nắm bắt và hiểu rõ tâm lý, hành vi của người tiêu dùng là vô cùng quan trọng để các doanh nghiệp có thể áp dụng các chiến lược tiếp thị hiệu quả. Việc đánh giá và dự đoán xu hướng thị trường cũng như tâm lý người tiêu dùng sẽ giúp các doanh nghiệp tạo ra các chiến dịch tiếp thị phù hợp và hiệu quả.

Trong thời kỳ khó khăn của nền kinh tế, người tiêu dùng thường có xu hướng cẩn trọng hơn trong việc chi tiêu. Họ có thể cắt giảm những khoản chi không cần thiết và tìm kiếm các sản phẩm, dịch vụ có giá trị thực sự. Do đó, các doanh nghiệp cần tập trung vào việc cải thiện chất lượng sản phẩm, tối ưu chi phí để đáp ứng nhu cầu của người tiêu dùng.

Xu hướng mua sắm thông minh và trải nghiệm mua hàng ngày càng trở nên phổ biến. Người tiêu dùng ngày càng thông minh hơn trong việc so sánh giá cả, tìm hiểu thông tin sản phẩm và đánh giá từ người tiêu dùng khác. Điều này đặt ra thách thức cho các doanh nghiệp trong việc cung cấp thông t

In [11]:
import semantic_kernel as sk

kernel = sk.Kernel()

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

api_key, org_id = sk.openai_settings_from_dot_env()
service_id = "default"
kernel.add_service(
    OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-0125", api_key=api_key, org_id=org_id),
)

plugin = kernel.import_plugin_from_prompt_directory("./Plugins/", "ReportPlugin")
Query_function = plugin["Query"]
Query = await kernel.invoke(Query_function, sk.KernelArguments(input="Tình hình hiện tại", big_title = "Báo cáo thị trường Tiêu dùng năm 2023"))
print(Query)


"Tình hình hiện tại thị trường tiêu dùng Việt Nam năm 2023"
